In [324]:
import dill
from fastai.tabular.all import *
from fastai.vision.all import *
from fastai.vision.utils import get_image_files

learn = load_learner(Path("/blue/hulcr/gmarais/Beetle_classifier/Models") / 'beetle_classifier3.pkl', cpu=False, pickle_module=dill)
learn.dls.vocab # get the classes in the model other than unknown
### Aplly to single image
# pred = learn.predict("/blue/hulcr/gmarais/Beetle_data/selected_images/test_data/Coccotypes_dactyliperda/Coccotypes_dactyliperda_16296_1_0052_1.JPG")
# print(pred)

### apply to multiple unlabelled images
# dataset_path=r"/blue/hulcr/gmarais/Beetle_data/selected_images"
# files = get_image_files(path=dataset_path, recurse=True, folders=('test_data'))
# apply_dl = learn.dls.test_dl(files, with_labels=False)
# preds, targets = learn.get_preds(dl=apply_dl)

['Coccotypes_dactyliperda', 'Hylesinus_varius', 'Phloeosinus_dentatus', 'Pityophthorus_juglandis', 'Platypus_cylindrus', 'Pycnarthrum_hispidium', 'Scolotodes_schwarzi', 'Xyleborus_affinis', 'Xylosandrus_compactus', 'Xylosandrus_crassiusculus']

In [59]:
import numpy as np

In [334]:
pred = learn.predict("/blue/hulcr/gmarais/Beetle_data/selected_images/test_data/Coccotypes_dactyliperda/Coccotypes_dactyliperda_16296_1_0052_1.JPG")
pred = np.array(pred[2])
pred

array([0.93305004, 0.00879687, 0.00597848, 0.00576069, 0.00848262,
       0.00607262, 0.00165288, 0.0157974 , 0.00234922, 0.01205918],
      dtype=float32)

In [413]:
pred_max = np.array([0.5,0.5,0.05,0.07,0.03,0,0,0])

In [433]:
# this function only describes how much a singular value in al ist stands out.
# if all values in the lsit are high or low this is 1
# the smaller the proportiopn of number of disimilar vlaues are to other more similar values the lower this number
# the larger the gap between the dissimilar numbers and the simialr number the smaller this number
# only able to interpret probabilities or values between 0 and 1
# this function outputs an estimate an inverse of the classification confidence based on the probabilities of all the classes.

def unkown_probs_calc(probs, threshold):
    # above threshold sqrt the probs and below threshold **2 the probs
    probs = probs - threshold
    diff_matrix = np.abs(probs[:, np.newaxis] - probs)
    diff_matrix_sum = np.sum(diff_matrix)
    probs_sum = np.sum(probs)
    class_val = (diff_matrix_sum/probs_sum)
    max_class_val = ((len(probs)-1)*2)
    kown_prob = class_val/max_class_val
    unknown_prob = 1-kown_prob
    
    return(unknown_prob)


unkown_probs_calc(probs=pred, threshold=0.9)

(0.05343659718831384, 0.09999999999999998)

In [439]:
np.sqrt(0.99)

0.99498743710662

In [437]:
0.01*0.01

0.0001

In [ ]:
# becasue the softmax output alsways sums to 1 we can use a different function too

def unkown_probs_calc_zero_sum(probs):
    
    return(unknown_prob)

In [ ]:
# ADD SEGMENTATION HERE AND ADD SCRIPT TO GRADIO
# ADD UNKNOWN CLASS APPLY FUNCTION